# Étape 4 — Jointures (Merge) et enrichissement des ventes

L’objectif de cette étape est d’enrichir les données de ventes (`order_lines`)
avec les informations issues des fichiers clients (`customers`) et produits
(`products`) afin de produire une table d’analyse unique.

Cette étape inclut la vérification des clés de jointure, le contrôle de la
qualité des merges, le recalcul de certaines variables business et une
mini-analyse après enrichissement.

**Import des librairies**

In [1]:
# Import des librairies nécessaires aux jointures et contrôles de qualité

import pandas as pd
import numpy as np

# Options d'affichage pour une meilleure lisibilité des DataFrames
pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", "{:,.2f}".format)

**Chargement des données nettoyées**

In [2]:
# Chargement des données nettoyées depuis le dossier data/processed

order_lines = pd.read_csv("../data/processed/order_lines_clean.csv")
customers = pd.read_csv("../data/processed/customers_clean.csv")
products = pd.read_csv("../data/processed/products_clean.csv")

# Vérification rapide des dimensions des jeux de données
print("order_lines :", order_lines.shape)
print("customers   :", customers.shape)
print("products    :", products.shape)

order_lines : (2188, 20)
customers   : (500, 6)
products    : (60, 5)


# Vérification des clés

**Vérification des clés clients (customer_id)**

In [3]:
# Vérification de la présence des clés customer_id dans les ventes et les clients

# Nombre de customer_id distincts dans chaque fichier
nb_customers_orders = order_lines["customer_id"].nunique()
nb_customers_customers = customers["customer_id"].nunique()

print(f"Nombre de clients distincts dans order_lines : {nb_customers_orders}")
print(f"Nombre de clients distincts dans customers   : {nb_customers_customers}")

# Identification des customer_id présents dans les ventes mais absents des clients
customers_manquants = set(order_lines["customer_id"]) - set(customers["customer_id"])

print(f"Nombre de customer_id présents dans les ventes mais absents des clients : {len(customers_manquants)}")

Nombre de clients distincts dans order_lines : 492
Nombre de clients distincts dans customers   : 500
Nombre de customer_id présents dans les ventes mais absents des clients : 0


**Vérification des clés produits (product_id)**

In [4]:
# Vérification de la présence des clés product_id dans les ventes et les produits

# Nombre de product_id distincts dans chaque fichier
nb_products_orders = order_lines["product_id"].nunique()
nb_products_products = products["product_id"].nunique()

print(f"Nombre de produits distincts dans order_lines : {nb_products_orders}")
print(f"Nombre de produits distincts dans products    : {nb_products_products}")

# Identification des product_id présents dans les ventes mais absents du catalogue produits
products_manquants = set(order_lines["product_id"]) - set(products["product_id"])

print(f"Nombre de product_id présents dans les ventes mais absents des produits : {len(products_manquants)}")

Nombre de produits distincts dans order_lines : 60
Nombre de produits distincts dans products    : 60
Nombre de product_id présents dans les ventes mais absents des produits : 0


# Les jointures

**Jointure ventes , clients**

In [5]:
#Jointure des ventes avec les informations clients
# Clé de jointure : customer_id
# Type de jointure : left 

orders_clients = order_lines.merge(customers,
                                    on= "customer_id",
                                    how= "left",
                                    suffixes=("","_customer")
                                    )

# Vérification du après jointure
orders_clients.shape

(2188, 25)

**Jointure résultat , produits(product_id, left join)**

In [6]:
# Jointure du jeu ventes-clients avec les informations produits
# Clé de jointure : product_id
# Type de jointure : left

orders_enriched = orders_clients.merge( products,
                                        on= "product_id",
                                        how= "left",
                                        suffixes=("", "_product")
                                      )

# Vérification de la dimension après jointure
orders_enriched.shape

(2188, 29)

# Contrôle de la qualité des jointures

**Controle qualité des jointures (taille avant/après)**

In [7]:
# Taille avant et après jointures

print("Taille orders_lines_clean : ", order_lines.shape)
print("Taille orders_clients : ", orders_clients.shape)
print("Taille orders_enriched:", orders_enriched.shape)

Taille orders_lines_clean :  (2188, 20)
Taille orders_clients :  (2188, 25)
Taille orders_enriched: (2188, 29)


In [8]:
# Vérification stricte : même nombre de lignes avant/après merge
order_lines.shape[0] == orders_enriched.shape[0]

True

**Lignes sans correspondance clients**

In [9]:
missing_client_city = orders_enriched["city"].isna().sum()
missing_client_segment = orders_enriched["segment"].isna().sum()

missing_client_city

0

**Lignes sans correspondance produit**

In [10]:
missing_product_category = orders_enriched["category"].isna().sum()
missing_product_price = orders_enriched["unit_price"].isna().sum()

**Tableau résumé**

In [11]:

control_summary = pd.DataFrame({
    "Problème détecté": [
        "Clients sans ville",
        "Clients sans segment",
        "Produits sans catégorie",
        "Produits sans prix unitaire"
    ],
    "Nombre de lignes": [
        missing_client_city,
        missing_client_segment,
        missing_product_category,
        missing_product_price
    ]
})

control_summary

,Problème détecté,Nombre de lignes
0,Clients sans ville,0
1,Clients sans segment,0
2,Produits sans catégorie,0
3,Produits sans prix unitaire,0


# Recalcul des colonnes business après enrichissement

**Sécurisation du type de discount_pct**

In [12]:
orders_enriched["discount_pct"] = pd.to_numeric(
    orders_enriched["discount_pct"],
    errors="coerce"
)

**Recalcul du montant brut**

In [13]:
orders_enriched["gross_amount_calc"] = (
    orders_enriched["unit_price"] * orders_enriched["quantity"]
)

**Recalcul du montant net**

In [14]:
orders_enriched["net_amount_calc"] = (
    orders_enriched["gross_amount_calc"] * (1 - orders_enriched["discount_pct"])
)

**Vérification des nouvelles colonnes**

In [15]:
orders_enriched[
    [
        "unit_price",
        "quantity",
        "discount_pct",
        "gross_amount",
        "gross_amount_calc",
        "net_amount",
        "net_amount_calc"
    ]
].head(40)

,unit_price,quantity,discount_pct,gross_amount,gross_amount_calc,net_amount,net_amount_calc
0,165.54,4,0.18,662.16,662.16,542.97,542.97
1,46.48,5,0.14,232.40,232.40,199.86,199.86
2,"1,123.49",2,0.04,"2,246.98","2,246.98","2,157.10","2,157.10"
3,68.31,2,0.22,136.62,136.62,106.56,106.56
4,"1,932.53",3,0.17,"5,797.59","5,797.59","4,812.00","4,812.00"
5,125.59,3,0.24,376.77,376.77,286.35,286.35
6,52.97,2,0.12,105.94,105.94,93.23,93.23
7,50.57,2,0.10,101.14,101.14,91.03,91.03
8,222.36,1,0.17,222.36,222.36,184.56,184.56
9,74.61,2,0.02,149.22,149.22,146.24,146.24


# Comparaison des montants et détection des lignes suspectes

**Calcul de l’écart entre montant observé et montant recalculé**

In [16]:
orders_enriched["amount_diff"] = (
    orders_enriched["net_amount"] - orders_enriched["net_amount_calc"]
)

orders_enriched

,order_id,customer_id,product_id,order_date,quantity,unit_price,discount_pct,gross_amount,net_amount,payment_method,channel,marketing_source,delivery_days,returned,review_score,segment,city,category,net_calc,amount_diff,age,gender,city_customer,segment_customer,signup_date,category_product,brand,product_name,unit_price_product,gross_amount_calc,net_amount_calc
0,O00001,C0201,P031,2024-06-16,4,165.54,0.18,662.16,542.97,Carte,App,Organique,1.00,0,5.00,Indépendant,Port-au-Prince,Cours,542.97,-0.00,32.00,F,Port-au-Prince,Indépendant,2025-01-20,Cours,DataCampX,Data Viz,165.54,662.16,542.97
1,O00002,C0141,P041,2024-06-10,5,46.48,0.14,232.40,199.86,Carte,Web,Réseaux sociaux,0.00,0,4.00,Étudiant,Abidjan,Livre,199.86,-0.00,39.00,M,Abidjan,Étudiant,2023-07-15,Livre,O'Reilly,Data Engineering,46.48,232.40,199.86
2,O00003,C0025,P005,2024-08-29,2,"1,123.49",0.04,"2,246.98","2,157.10",Virement,Web,Réseaux sociaux,3.00,0,4.00,Étudiant,Cap-Haïtien,Laptop,"2,157.10",-0.00,37.00,F,Cap-Haïtien,Étudiant,2024-01-12,Laptop,HP,Gaming 16,"1,123.49","2,246.98","2,157.10"
3,O00004,C0188,P042,2024-01-27,2,68.31,0.22,136.62,106.56,Carte,Boutique,Partenariat,0.00,0,4.00,Indépendant,Abidjan,Cloud,106.56,-0.00,18.00,M,Abidjan,Indépendant,2025-01-07,Cloud,Azure,Crédit 500$,68.31,136.62,106.56
4,O00005,C0023,P052,2025-08-06,3,"1,932.53",0.17,"5,797.59","4,812.00",Carte,Web,Référencement payant,9.00,0,5.00,Professionnel,Cap-Haïtien,Laptop,"4,812.00",0.00,25.00,Unknown,Cap-Haïtien,Professionnel,2025-07-04,Laptop,HP,Ultrabook 14,"1,932.53","5,797.59","4,812.00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2183,O02196,C0378,P004,2024-02-02,2,91.27,0.05,182.54,173.41,Mobile Money,Web,Organique,3.00,0,5.00,Entreprise,Dakar,Accessoire,173.41,-0.00,24.00,M,Dakar,Entreprise,2024-02-12,Accessoire,Anker,SSD externe,91.27,182.54,173.41
2184,O02197,C0046,P047,2025-07-01,2,122.90,0.12,245.80,216.30,Mobile Money,Web,Organique,2.00,0,5.00,Entreprise,Saint-Marc,Cours,216.30,-0.00,31.00,M,Saint-Marc,Entreprise,2023-07-26,Cours,DeepLearners,SQL avancé,122.90,245.80,216.30
2185,O02198,C0126,P002,2025-03-23,3,"1,058.14",0.16,"3,174.42","2,666.51",Mobile Money,Web,Référencement payant,15.00,1,4.00,Professionnel,Jacmel,Laptop,"2,666.51",-0.00,23.00,F,Jacmel,Professionnel,2024-07-25,Laptop,ASUS,Notebook 15,"1,058.14","3,174.42","2,666.51"
2186,O02199,C0303,P026,2025-07-20,1,95.37,0.11,95.37,84.88,Carte,Web,Email,13.00,0,5.00,Indépendant,Paris,Accessoire,84.88,0.00,28.00,M,Paris,Indépendant,2025-12-07,Accessoire,Generic,Clavier,95.37,95.37,84.88


**Identification des lignes suspectes**

In [17]:
# une ligne est suspecte si ∣amount_diff∣>0.01

suspect_orders = orders_enriched[
    orders_enriched["amount_diff"].abs() > 0.01
]


**Nombre de lignes suspectes**

In [18]:
suspect_orders.shape[0]

0

# Mini-analyse

**Création du tableau croisé**

In [19]:
# Vision globale des colonnes du data_set orders_enriched
orders_enriched.columns

Index(['order_id', 'customer_id', 'product_id', 'order_date', 'quantity',
       'unit_price', 'discount_pct', 'gross_amount', 'net_amount',
       'payment_method', 'channel', 'marketing_source', 'delivery_days',
       'returned', 'review_score', 'segment', 'city', 'category', 'net_calc',
       'amount_diff', 'age', 'gender', 'city_customer', 'segment_customer',
       'signup_date', 'category_product', 'brand', 'product_name',
       'unit_price_product', 'gross_amount_calc', 'net_amount_calc'],
      dtype='object')

In [20]:
# Création du tableau croisé (Segment en ligne, Catégorie en colonne)
ca_segment_category = pd.pivot_table(
    orders_enriched,
    values="net_amount_calc",
    index="segment_customer",
    columns="category_product",
    aggfunc="sum"
)


In [21]:
# Calcul du Total par segment (Lignes)
ca_segment_category["Total_CA"] = ca_segment_category.sum(axis=1)

# Calcul du Total par catégorie (Colonnes)
total_row = ca_segment_category.sum(axis=0)
total_row.name = "TOTAL_GLOBAL"



In [22]:
# Assemblage et tri 
ca_segment_category = pd.concat([
    ca_segment_category.sort_values(by="Total_CA", ascending=False),
    total_row.to_frame().T
])

# Affichage de segment_customer en ligne
ca_segment_category.index.name = "segment_customer"

ca_segment_category


category_product,Accessoire,Cloud,Cours,Laptop,Livre,Logiciel,Total_CA
segment_customer,,,,,,,
Étudiant,"16,269.22","49,248.83","77,883.99","564,245.03","13,493.36","52,649.34","773,789.78"
Professionnel,"18,279.29","56,899.33","69,387.14","507,813.58","15,235.30","58,983.64","726,598.28"
Indépendant,"8,542.39","23,821.76","35,781.22","193,132.43","7,845.75","31,424.59","300,548.14"
Entreprise,"7,435.85","25,003.17","20,136.92","145,515.61","4,128.03","20,502.74","222,722.33"
TOTAL_GLOBAL,"50,526.75","154,973.09","203,189.26","1,410,706.66","40,702.45","163,560.32","2,023,658.53"


**Export du tableau du chiffre d'affaires par catégorie par segment vers results/tables**

In [23]:
ca_segment_category.to_csv("../results/tables/ca_segment_categorie.csv", index=False)

**Suppression des colonnes redondantes dans orders_enriched**

In [24]:
# Suppression des colonnes redondantes
columns_to_drop = ["city", "segment", "category", "unit_price"]
orders_enriched_clean = orders_enriched.drop(columns=columns_to_drop)

# Harmonisation des noms de colonnes
orders_enriched_clean = orders_enriched_clean.rename(columns={
    "city_customer": "city",
    "segment_customer": "segment",
    "category_product": "category",
    "unit_price_product": "unit_price"
})


In [25]:
orders_enriched_clean.shape

(2188, 27)

**Export de orders_enriched.csv**

In [26]:
orders_enriched_clean.to_csv( "../results/tables/orders_enriched.csv", index=False)
print("Export terminé : orders_enriched.csv enregistré dans results/tables/")

Export terminé : orders_enriched.csv enregistré dans results/tables/


# Commentaires

La jointure des ventes avec les tables clients et produits n’a entraîné aucune perte d’information, comme l’indiquent les tailles identiques avant et après merge ainsi que l’absence de `customer_id` et `product_id` non appariés.  
Les contrôles post-jointure montrent qu’aucune vente ne présente de valeurs manquantes pour les variables clés (ville, segment, catégorie, prix unitaire), confirmant la qualité des clés de jointure.  
Le recalcul des montants financiers n’a révélé aucune incohérence, aucune ligne suspecte n’étant détectée entre les montants nets observés et recalculés.  
Le tableau croisé met en évidence une forte concentration du chiffre d’affaires sur la catégorie *Laptop*, tous segments confondus, avec une contribution dominante des segments *Étudiant* et *Professionnel*.  
Le chiffre d’affaires total issu du tableau agrégé (2 023 658,53) est cohérent avec la somme des ventes nettes recalculées, validant la fiabilité de la table enrichie finale.
